# Add trial labels

Brief 1-2 sentence description of notebook.

In [12]:
import os
import glob
import git
import sys


In [13]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [14]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [15]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [16]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [17]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [66]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

# VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention/results/2024_01_30_rce_2nd_cohort/proc/rce_pilot_2_04_lfp_spectral_sleap_spikes.pkl")
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs



In [67]:
TRIAL_LABELS_DF = pd.read_excel("./rce_per_trial_labeling.xlsx")
TRIAL_LABELS_DF = TRIAL_LABELS_DF.rename(columns={col: col.strip().lower() for col in TRIAL_LABELS_DF.columns})
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition"])
TRIAL_LABELS_DF["time_stamp_index"] = TRIAL_LABELS_DF["time_stamp_index"].astype(np.int64)


In [68]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.drop_duplicates(subset=["session_dir", "time_stamp_index", 'video_file'])

In [69]:
TRIAL_LABELS_DF["video_file"].unique()

array(['20221202_134600_omission_and_competition_subject_6_1_and_6_2.1.videoTimeStamps.cameraHWSync',
       '20221203_154800_omission_and_competition_subject_6_4_and_6_1.1.videoTimeStamps.cameraHWSync',
       '20221214_125409_om_and_comp_6_1_and_6_3.1.videoTimeStamps.cameraHWSync',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1.videoTimeStamps.cameraHWSync',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1.videoTimeStamps.cameraHWSync',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.2.videoTimeStamps.cameraHWSync',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.videoTimeStamps.cameraHWSync',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1.videoTimeStamps.cameraHWSync',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1.videoTimeStamps.cameraHWSync',
       '20230617

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [70]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [71]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_trial_labels.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [72]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.216, 0.633, 0.282, 0.613, 0.636, 1.501, 0....","[[0.39, 0.2312, 0.02298, 0.1543, 0.315, 0.7856...","[[1.783, 1.615, 1.699, 0.9365, 1.126, 1.648, 0...","[[1.663, 0.849, 1.167, 0.758, 0.328, 0.7144, 1...","[[2.055, 0.452, 0.3164, 0.5933, 0.8477, 0.6006...","[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[1.216, 0.633, 0.282, 0.613, 0.636, 1.501, 0....","[[0.39, 0.2312, 0.02298, 0.1543, 0.315, 0.7856...","[[1.783, 1.615, 1.699, 0.9365, 1.126, 1.648, 0...","[[1.663, 0.849, 1.167, 0.758, 0.328, 0.7144, 1...","[[2.055, 0.452, 0.3164, 0.5933, 0.8477, 0.6006...","[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.848, 1.116, 0.682, 0.198, 0.154, 0.127, 0....","[[1.97, 1.198, 0.10645, 0.2512, 0.3628, 0.595,...","[[0.09326, 0.08734, 0.136, 0.0357, 0.1394, 0.2...","[[1.11, 0.2117, 0.3718, 0.1334, 0.174, 0.7544,...","[[1.734, 1.35, 0.8564, 0.2327, 0.08453, 0.0556...","[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[2.848, 1.116, 0.682, 0.198, 0.154, 0.127, 0....","[[1.97, 1.198, 0.10645, 0.2512, 0.3628, 0.595,...","[[0.09326, 0.08734, 0.136, 0.0357, 0.1394, 0.2...","[[1.11, 0.2117, 0.3718, 0.1334, 0.174, 0.7544,...","[[1.734, 1.35, 0.8564, 0.2327, 0.08453, 0.0556...","[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.

In [73]:
TRIAL_LABELS_DF.shape

(818, 16)

In [74]:
# grouped = TRIAL_LABELS_DF.groupby(['recording_file', 'video_file'])['time_stamp_index'].apply(lambda x: sorted(list(x))).reset_index()


grouped = TRIAL_LABELS_DF.groupby(['session_dir', 'video_file']).agg({'time_stamp_index': lambda x: list(x), 'condition': lambda x: list(x), 'competition_closeness': lambda x: list(x), 'video_frame': lambda x: list(x)}).reset_index()


In [75]:
grouped.head()

,session_dir,video_file,time_stamp_index,condition,competition_closeness,video_frame
0,20221202_134600_omission_and_competition_subje...,20221202_134600_omission_and_competition_subje...,"[1390826, 2990825, 4790823, 6390821, 7890820, ...","[rewarded, rewarded, rewarded, omission, rewar...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1734.0, 3728.0, 5972.0, 7966.0, 9836.0, 12329..."
1,20221203_154800_omission_and_competition_subje...,20221203_154800_omission_and_competition_subje...,"[1323915, 2923914, 4723911, 6323909, 7823908, ...","[rewarded, rewarded, rewarded, omission, rewar...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1650.0, 3645.0, 5889.0, 7883.0, 9752.0, 12246..."
2,20221214_125409_om_and_comp_6_1_and_6_3,20221214_125409_om_and_comp_6_1_and_6_3.1.vide...,"[1408048, 3008051, 4808043, 6408042, 7908040, ...","[rewarded, rewarded, rewarded, omission, rewar...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1405.0, 3002.0, 4798.0, 6395.0, 7892.0, 9889...."
3,20221215_145401_comp_amd_om_6_1_and_6_3,20221215_145401_comp_amd_om_6_1_and_6_3.1.vide...,"[1872697, 3472694, 5272689, 6872686, 8372686, ...","[6.1, 6.1, 6.1, 6.1, 6.1, 6.1, 6.1, 6.1, 6.1, ...","[Subj 1 Only, Subj 1 Only, Subj 1 Only, Subj 1...","[2387.0, 4382.0, 6626.0, 8620.0, 10489.0, 1298..."
4,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[982229, 3382227, 5682225, 7482224, 8582220, 9...","[1.4, 1.3, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.3, ...","[Subj 2 Only, Subj 1 Only, Subj 2 Only, Subj 2...","[980.0, 3376.0, 5671.0, 7468.0, 8566.0, 9564.0..."


In [77]:
grouped.shape

(26, 6)

In [78]:
grouped["time_stamp_index"].iloc[0]

[1390826,
 2990825,
 4790823,
 6390821,
 7890820,
 9890818,
 11790816,
 13590815,
 15190821,
 16990812,
 18990809,
 20790805,
 23190805,
 24990804,
 30949197,
 32549196,
 34349195,
 35949193,
 37449192,
 39449187,
 41349188,
 43149186,
 44749185,
 46549183,
 48549181,
 50349180,
 52749175,
 54549173,
 56249171,
 58049170,
 59949171,
 62349168,
 63949167]

In [81]:
grouped["video_file"] = grouped["video_file"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))
VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [82]:
grouped["video_file"].unique()

array(['20221202_134600_omission_and_competition_subject_6_1_and_6_2.1',
       '20221203_154800_omission_and_competition_subject_6_4_and_6_1.1',
       '20221214_125409_om_and_comp_6_1_and_6_3.1',
       '20221215_145401_comp_amd_om_6_1_and_6_3.1',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.2',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4',
       '20

In [83]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["video_name"].unique()

array(['20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1',
       '20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.2',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1',
       '20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.2',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1',
       '20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.1',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.2',
       '20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2.3',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.1',
       '20230616_111904_standard_comp_to_training_D4_subj_1-4_and_1-2.2',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.2',
       '20230617_115521_standard_comp_

In [84]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped, how='inner', left_on=['session_dir', 'video_name'], right_on=['session_dir', 'video_file'])

In [87]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF.rename(columns={'time_stamp_index': 'trial_time_stamps', 'condition': 'trial_label', 'competition_closeness': 'trial_competition_closeness', "video_frame": "trial_frame"}, errors="ignore")

In [88]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies,video_file,trial_time_stamps,trial_label,trial_competition_closeness,trial_frame
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230612_101430_standard_comp_to_training_D1_s...,"[982229, 3382227, 5682225, 7482224, 8582220, 9...","[1.4, 1.3, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.3, ...","[Subj 2 Only, Subj 1 Only, Subj 2 Only, Subj 2...","[980.0, 3376.0, 5671.0, 7468.0, 8566.0, 9564.0..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230612_101430_standard_comp_to_training_D1_s...,"[982229, 3382227, 5682225, 7482224, 8582220, 9...","[1.4, 1.3, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.3, ...","[Subj 2 Only, Subj 1 Only, Subj 2 Only, Subj 2...","[980.0, 3376.0, 5671.0, 7468.0, 8566.0, 9564.0..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230612_101430_standard_comp_to_training_D1_s...,"[982229, 3382227, 5682225, 7482224, 8582220, 9...","[1.4, 1.3, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.3, ...","[Subj 2 Only, Subj 1 Only, Subj 2 Only, Subj 2...","[980.0, 3376.0, 5671.0, 7468.0, 8566.0, 9564.0..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230612_101430_standard_comp_to_training_D1_s...,"[982229, 3382227, 5682225, 7482224, 8582220, 9...","[1.4, 1.3, 1.4, 1.4, 1.4, 1.3, 1.4, 1.3, 1.3, ...","[Subj 2 Only, Subj 1 Only, Subj 2 Only, Subj 2...","[980.0, 3376.0, 5671.0, 7468.0, 8566.0, 9564.0..."
4,2,20230612_112630_standard_c

In [90]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_time_stamps"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: np.array(x))
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_frame"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_frame"].apply(lambda x: np.array(x))

In [91]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].iloc[0]

array(['1.4', '1.3', '1.4', '1.4', '1.4', '1.3', '1.4', '1.3', '1.3',
       '1.3', '1.3', '1.3', '1.3', '1.4', '1.3', '1.4', '1.3', '1.3',
       '1.3', '1.3', 'rewarded', 'rewarded', 'rewarded', 'rewarded',
       'rewarded', 'rewarded', 'rewarded', 'rewarded', 'rewarded',
       'rewarded', 'rewarded', 'rewarded', 'rewarded', 'rewarded',
       'rewarded', 'rewarded', 'rewarded', 'rewarded', 'rewarded',
       'rewarded'], dtype='<U32')

In [92]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].iloc[0]

array(['Subj 2 Only', 'Subj 1 Only', 'Subj 2 Only', 'Subj 2 Only',
       'Subj 2 Only', 'Subj 1 Only', 'Subj 2 Only',
       'Subj 1 blocking Subj 2', 'Subj 1 blocking Subj 2', 'Subj 1 Only',
       'Subj 1 Only', 'Subj 1 blocking Subj 2', 'Subj 1 blocking Subj 2',
       'Subj 2 blocking Subj 1', 'Subj 1 Only', 'Subj 2 Only',
       'Subj 1 blocking Subj 2', 'Subj 1 blocking Subj 2',
       'Subj 1 then Subj 2', 'Subj 1 then Subj 2', 'nan', 'nan', 'nan',
       'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan',
       'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'],
      dtype='<U32')

- Converting the trial label to win or lose based on who won the trial

In [21]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
     

In [95]:
MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"] = MERGED_VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_label"].apply(
    lambda x: ["win" if str(label).strip() == str(x["subject"]).strip() else "lose" if str(label).strip() in x["all_subjects"] else label for label in x["trial_label"]], 
    axis=1)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

- Adding the competition closeness as a column

In [32]:
all_competition_closeness = np.unique(np.concatenate(VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"]))

In [24]:
raise ValueError()

ValueError: 

In [39]:
# competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else "comp" if type(k) is str else np.nan for k in VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].iloc[0]}
competition_closeness_map = {k: "non_comp" if "only" in str(k).lower() else ("comp" if "subj" in k.lower() else np.nan) for k in all_competition_closeness}

In [40]:
competition_closeness_map

{'Subj 1 Only': 'non_comp',
 'Subj 1 blocking Subj 2': 'comp',
 'Subj 1 then Subj 2': 'comp',
 'Subj 2 Only': 'non_comp',
 'Subj 2 blocking Subj 1': 'comp',
 'Subj 2 then Subj 1': 'comp',
 'nan': nan}

In [42]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"] = VIDEO_TO_FRAME_AND_SUBJECT_DF["trial_competition_closeness"].apply(lambda x: [competition_closeness_map[label] for label in x])

In [43]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,video_file,trial_time_stamps,trial_label,trial_competition_closeness
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[lose, win, win, lose, win, lose, lose, lose, ...","[non_comp, non_comp, non_comp, non_comp, non_c..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",20230618_100636_standard_comp_to_omission_D2_s...,"[1099250, 3499256, 5799251, 7599250, 8699250, ...","[win, lose, lose, win, lose, win, win, win, lo...","[non_comp, non_comp, non_comp, non_comp, non_c..."


In [44]:
VIDEO_TO_FRAME_AND_SUBJECT_DF

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
     

In [45]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle("./proc/rce_pilot_2_05_lfp_spectral_sleap_spikes_labels.pkl")

In [ ]:
raise ValueError("This is not done yet")

In [ ]:
ALL_PHY_DIR

In [ ]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

In [ ]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

- Combining all the unit info dataframes and adding the recording name

In [ ]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [ ]:
recording_to_cluster_info_df.head()

- Filtering for the good units

In [ ]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [ ]:
good_unit_cluster_info_df.head()

In [ ]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [ ]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

In [ ]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

- The times that all the spikes happened

In [ ]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

In [ ]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

### Combining everything into a dataframe

In [ ]:
recording_to_spike_df = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

In [ ]:
cluster_info_df.head()

In [ ]:
spike_times

In [ ]:
spike_clusters

- Combining the spike time df for all recordings

In [ ]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [ ]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [ ]:
all_spike_time_df.head()

In [ ]:
all_spike_time_df.tail()

## Grouping all the neurons by recording

In [ ]:
# Grouping all spike times by neuron and recording

grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: sorted(list(x))).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: list(x)}).reset_index()

In [ ]:
grouped_df

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF = pd.merge(VIDEO_TO_FRAME_AND_SUBJECT_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

## Calculating the firing rates

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.columns

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.head()

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["start_stop_timestamps"][1], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"] = VIDEO_TO_FRAME_AND_SUBJECT_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,:10]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,10:20]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5,20:30]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.iloc[:5, 30:]

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_timestamps"].iloc[0].shape

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0].shape

In [ ]:
np.max(VIDEO_TO_FRAME_AND_SUBJECT_DF["neuron_average_fr"].iloc[0])

In [ ]:
VIDEO_TO_FRAME_AND_SUBJECT_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError("")